In [37]:
#data cleaning
with open('/content/drive/MyDrive/peterpan.txt') as f:
  peterpan_txt=f.read()


In [38]:
#more cleaning
# define punctuation
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

# remove punctuation from the string
no_punct = ""
for char in peterpan_txt:
   if char not in punctuations:
       no_punct = no_punct + char

In [39]:
peterpan_final=no_punct.lower()
peterpan_final

'all children except one grow up they soon know that they will grow up and the way wendy knew was this one day when she was two years old she was playing in a garden and she plucked another flower and ran with it to her mother i suppose she must have looked rather delightful for mrs darling put her hand to her heart and cried “oh why cant you remain like this for ever” this was all that passed between them on the subject but henceforth wendy knew that she must grow up you always know after you are two two is the beginning of the end\nof course they lived at 14 their house number on their street and until wendy came her mother was the chief one she was a lovely lady with a romantic mind and such a sweet mocking mouth her romantic mind was like the tiny boxes one within the other that come from the puzzling east however many you discover there is always one more and her sweet mocking mouth had one kiss on it that wendy could never get though there it was perfectly conspicuous in the righ

In [60]:
len(peterpan_final)

245936

In [41]:
#overwrite the content of peterpan
text_file = open("/content/drive/MyDrive/peterpan.txt", "w")
n = text_file.write(peterpan_final)
text_file.close()

In [42]:
# activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [43]:
#import packages

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType

# we are going to use this to time our queries.
import time

# Create a SparkSession
spark = SparkSession.builder.appName("PySpark").getOrCreate()
from pyspark import SparkFiles

In [52]:

paperpan_df = spark.read.text('/content/drive/MyDrive/peterpan.txt')


In [ ]:

paperpan_data = spark.sparkContext.textFile('/content/drive/MyDrive/peterpan.txt')
paperpan_data_split = paperpan_data.flatMap(lambda line: line.split(" "))


paperpan_data_split.collect()

In [54]:
paperpan_data_cnt = paperpan_data_split.map(lambda word: (word, 1))

In [55]:

from operator import add
paperpan_data_cnt_all = paperpan_data_cnt.reduceByKey(add)


In [58]:
paperpan_data_cnt_all.saveAsTextFile('peterpan_wordcount.txt')

In [59]:
spark.read.text('peterpan_wordcount.txt').show()

+----------------+
|           value|
+----------------+
|    ('all', 240)|
|('children', 86)|
|  ('except', 19)|
|    ('one', 206)|
|     ('grow', 8)|
|     ('up', 144)|
|   ('they', 569)|
|    ('soon', 30)|
|   ('that', 582)|
|    ('will', 82)|
|   ('and', 1361)|
|   ('the', 2303)|
|  ('wendy', 282)|
|    ('knew', 68)|
|     ('day', 18)|
|    ('she', 574)|
|     ('old', 25)|
| ('playing', 11)|
|      ('a', 921)|
| ('another', 33)|
+----------------+
only showing top 20 rows

